In [2]:
## Standard libraries
import os
import json
import math
import time
import numpy as np
import scipy.linalg

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint


# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "/local/rcs/yunyun"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "/local/rcs/yunyun/SSDG-main/resnet50.pth"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

/tmp/ipykernel_839558/1165486641.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
Global seed set to 42


Using device cuda:0


In [3]:
# Load CNN architecture pretrained on ImageNet
# os.environ["TORCH_HOME"] = CHECKPOINT_PATH
pretrained_model = torchvision.models.resnet50(pretrained=True)
pretrained_model = pretrained_model.to(device)

if CHECKPOINT_PATH:
    if os.path.isfile(CHECKPOINT_PATH):
        checkpoint = torch.load(CHECKPOINT_PATH) 
        # model.load_state_dict(checkpoint['state_dict'])  #['state_dict']
        pretrained_model.load_state_dict(checkpoint)  #['state_dict']
        print("=> load chechpoint found at {}".format(CHECKPOINT_PATH))
        # print("=> loaded checkpoint '{}' (epoch {})"
        #       .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(CHECKPOINT_PATH))
# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

=> load chechpoint found at /local/rcs/yunyun/SSDG-main/resnet50.pth


In [4]:
from torchsummary import summary
summary(pretrained_model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [51]:
# Mean and Std from ImageNet
# NORM_MEAN = np.array([0.485, 0.456, 0.406])
# NORM_STD = np.array([0.229, 0.224, 0.225])
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256), transforms.CenterCrop(224),
#     transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
])

In [52]:
import torchvision.datasets as dsets  
def image_folder_custom_label(root, transform, idx2label) :
    
    # custom_label
    # type : List
    # index -> label
    # ex) ['tench', 'goldfish', 'great_white_shark', 'tiger_shark']
    
    old_data = dsets.ImageFolder(root=root, transform=transform)
    old_classes = old_data.classes
    
    label2idx = {}
    
    for i, item in enumerate(idx2label) :
 
        label2idx[item] = old_classes[i]
    
    key_list = list(label2idx.keys())
    val_list = list(label2idx.values())
    
    new_data = dsets.ImageFolder(root=root, transform=transform, 
                                 target_transform=lambda x :  key_list[val_list.index(old_classes[x])])
    new_data.classes = idx2label
    new_data.class_to_idx = label2idx

    return new_data

In [53]:
# https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_idx = json.load(open("./imagenet_class_index.json"))
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [54]:
from utils import imshow
# Load dataset and create data loader
imagenet_path = os.path.join(DATASET_PATH, "ImageNet-Data")
assert os.path.isdir(imagenet_path), f"Could not find the ImageNet dataset at expected path \"{imagenet_path}\". " + \
                                     f"Please make sure to have downloaded the ImageNet dataset here, or change the {DATASET_PATH=} variable."

val_dataset = torchvision.datasets.ImageFolder(root=os.path.join(imagenet_path, 'val'), transform=plain_transforms)
val_dataset_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=8)

imagenet_data = image_folder_custom_label(root=os.path.join(imagenet_path, 'val'), transform=plain_transforms, idx2label=idx2label)
data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=32, shuffle=False)

# print(np.unique(val_dataset.targets))
# # Load label names to interpret the label numbers 0 to 999
# with open(os.path.join(imagenet_path, "label_list.json"), "r") as f:
#     label_names = json.load(f)

# def get_label_index(lab_str):
#     assert lab_str in label_names, f"Label \"{lab_str}\" not found. Check the spelling of the class."
#     return label_names.index(lab_str)

In [55]:
# images, labels = iter(data_loader).next()
# print(images.shape)
# print("True Image & True Label")
# imshow(torchvision.utils.make_grid(images, normalize=True), labels)

In [56]:
def eval_model(dataset_loader, img_func=None):
    tp, tp_5, counter = 0., 0., 0.
    for imgs, labels in tqdm(dataset_loader, desc="Validating..."):
        imgs = imgs.to(device)
        labels = labels.to(device)
        if img_func is not None:
            imgs = img_func(imgs, labels)
        with torch.no_grad():
            preds = pretrained_model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item()/counter
    top5 = tp_5.float().item()/counter
    print(f"Top-1 acc: {(100.0 * (acc)):4.2f}%")
    print(f"Top-5 acc: {(100.0 * (top5)):4.2f}%")
    return acc, top5

In [64]:
_ = eval_model(val_dataset_loader)

Validating...:   0%|          | 0/1563 [00:00<?, ?it/s]

Top-1 acc: 75.41%
Top-5 acc: 92.54%


In [11]:
from torchattacks import *

In [57]:
atks = [
    FGSM(pretrained_model, eps=4/255)
#     FGSM(pretrained_model, eps=8/255),
#     FGSM(pretrained_model, eps=16/255),
#     BIM(pretrained_model, eps=8/255, alpha=2/255, steps=10),
#     RFGSM(pretrained_model, eps=8/255, alpha=2/255, steps=100),
#     CW(pretrained_model, c=1, lr=0.01, steps=10, kappa=0),
#     PGD(pretrained_model, eps=8/255, alpha=2/225, steps=10,random_start=False)
#     PGDL2(pretrained_model, eps=1, alpha=0.2, steps=10)
#     EOTPGD(pretrained_model, eps=8/255, alpha=2/255, steps=100, eot_iter=2),
#     FFGSM(pretrained_model, eps=8/255, alpha=10/255),
#     TPGD(pretrained_model, eps=8/255, alpha=2/255, steps=100),
#     MIFGSM(pretrained_model, eps=8/255, alpha=2/255, steps=100, decay=0.1),
#     VANILA(pretrained_model),
#     GN(pretrained_model, std=0.1),
#     APGD(pretrained_model, eps=8/255, steps=100, eot_iter=1, n_restarts=1, loss='ce'),
#     APGD(pretrained_model, eps=8/255, steps=10, eot_iter=1, n_restarts=1, loss='dlr'),
#     APGDT(pretrained_model, eps=8/255, steps=100, eot_iter=1, n_restarts=1),
#     FAB(pretrained_model, eps=8/255, steps=100, n_classes=10, n_restarts=1, targeted=False),
#     FAB(pretrained_model, eps=8/255, steps=100, n_classes=10, n_restarts=1, targeted=True),
#     Square(pretrained_model, eps=8/255, n_queries=5000, n_restarts=1, loss='ce'),
#     AutoAttack(pretrained_model, eps=8/255, n_classes=10, version='standard'),
#     OnePixel(pretrained_model, pixels=5, steps=10, inf_batch=50),
#     DeepFool(pretrained_model, steps=10),
#     DIFGSM(pretrained_model, eps=8/255, alpha=2/255, steps=100, diversity_prob=0.5, resize_rate=0.9)
#       Pixle(pretrained_model, x_dimensions=(0.1, 0.2), restarts=10, max_iterations=10)
]

In [58]:
#     FGSM(pretrained_model, eps=8/255),
#     FGSM(pretrained_model, eps=16/255),
#     BIM(pretrained_model, eps=8/255, alpha=2/255, steps=10),
#     CW(pretrained_model, c=1, lr=0.01, steps=10, kappa=0),
#     PGD(pretrained_model, eps=8/255, alpha=2/225, steps=10, random_start=True)
#     PGD(pretrained_model, eps=16/255, alpha=2/225, steps=10, random_start=True)
#     PGDL2(pretrained_model, eps=1, alpha=0.2, steps=10)

In [59]:
class_idx[str(10)][0]

'n01530575'

In [60]:
idx2label

['tench',
 'goldfish',
 'great_white_shark',
 'tiger_shark',
 'hammerhead',
 'electric_ray',
 'stingray',
 'cock',
 'hen',
 'ostrich',
 'brambling',
 'goldfinch',
 'house_finch',
 'junco',
 'indigo_bunting',
 'robin',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water_ouzel',
 'kite',
 'bald_eagle',
 'vulture',
 'great_grey_owl',
 'European_fire_salamander',
 'common_newt',
 'eft',
 'spotted_salamander',
 'axolotl',
 'bullfrog',
 'tree_frog',
 'tailed_frog',
 'loggerhead',
 'leatherback_turtle',
 'mud_turtle',
 'terrapin',
 'box_turtle',
 'banded_gecko',
 'common_iguana',
 'American_chameleon',
 'whiptail',
 'agama',
 'frilled_lizard',
 'alligator_lizard',
 'Gila_monster',
 'green_lizard',
 'African_chameleon',
 'Komodo_dragon',
 'African_crocodile',
 'American_alligator',
 'triceratops',
 'thunder_snake',
 'ringneck_snake',
 'hognose_snake',
 'green_snake',
 'king_snake',
 'garter_snake',
 'water_snake',
 'vine_snake',
 'night_snake',
 'boa_constrictor',
 'rock_python',
 'Indian_cobr

In [61]:
def imsave(img, img_path):
    npimg = img.numpy()
#     plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.imsave(img_path, np.transpose(npimg,(1,2,0)))

In [62]:
# inv_transforms = torch.nn.Sequential(
#             transforms.Normalize(mean=[ 0., 0., 0. ], std=[ 1/0.229, 1/0.224, 1/0.225 ]),
#             transforms.Normalize(mean=[ -0.485, -0.456, -0.406 ], std=[ 1., 1., 1.] )
#         )
# denormalize = torch.jit.script(inv_transforms)
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from torchvision.transforms import ToPILImage
val_dataset = torchvision.datasets.ImageFolder(root=os.path.join(imagenet_path, 'val'), transform=plain_transforms)
val_dataset_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False, num_workers=8)

In [63]:
IMAGENET_DEFAULT_MEAN

(0.485, 0.456, 0.406)

In [64]:
IMAGENET_DEFAULT_STD
std = torch.as_tensor(IMAGENET_DEFAULT_STD).to(device)[None, :, None, None]
print(std)

tensor([[[[0.2290]],

         [[0.2240]],

         [[0.2250]]]], device='cuda:0')


In [71]:
print("Adversarial Image & Predicted Label")
save_root_path = '/local/rcs/yunyun/ImageNet-Data/Attack'

for atk in atks :
    
    print("-"*70)
    print(atk)
    
    save_attack_folder = os.path.join(save_root_path, 'fgsm_4')
    
    correct = 0
    total = 0
    start = time.time()
    count = 0
#     with tqdm(val_dataset_loader, unit=" batch") as tepoch:
#         for images, labels in tepoch:
    for i, (images, labels) in enumerate(tqdm(val_dataset_loader)):

        labels = labels.to(device)
        cls_idx = class_idx[str(labels.item())][0]

        folderExists = os.path.exists(os.path.join(save_attack_folder, cls_idx))
        if folderExists == False:
            os.mkdir(os.path.join(save_attack_folder, cls_idx))

        adv_images = atk(images, labels)

        outputs = pretrained_model(adv_images)

        _, pre = torch.max(outputs.data, 1)

        total += images.shape[0]
        correct += (pre == labels).sum()

        img_file_path = os.path.join(save_attack_folder, cls_idx) + '/' + cls_idx + '_' + str(i).zfill(5) + '.JPEG'

    #         mean = torch.as_tensor(IMAGENET_DEFAULT_MEAN).to(device)[None, :, None, None]
    #         std = torch.as_tensor(IMAGENET_DEFAULT_STD).to(device)[None, :, None, None]
    #         unnorm_images = adv_images * std + mean
    #         print(adv_images.max())
    #         print(adv_images.min())
    #         print(unnorm_images.shape)
#             pil_img = ToPILImage()(adv_images[0, :])
#             pil_img.save(img_file_path)
        imsave(adv_images.cpu().data[0], img_file_path)
        count += 1
        tepoch.set_postfix(accuracy=(correct.item()/total))


    #         imshow(torchvision.utils.make_grid(adv_images.cpu().data, normalize=True), [imagnet_data.classes[i] for i in pre])

print('Total elapsed time (sec): %.2f' % (time.time() - start))
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Adversarial Image & Predicted Label
----------------------------------------------------------------------
FGSM(model_name=ResNet, device=cuda:0, eps=0.01568627450980392, attack_mode=default, return_type=float)


  0%|          | 0/50000 [00:00<?, ?it/s]

Total elapsed time (sec): 2176.46
Robust accuracy: 1.55 %


In [ ]:
print(str(1).zfill(5))

In [68]:
# Mean and Std from ImageNet
# NORM_MEAN = np.array([0.485, 0.456, 0.406])
# NORM_STD = np.array([0.229, 0.224, 0.225])
# from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
# No resizing and center crop necessary as images are already preprocessed.
unorm_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256), transforms.CenterCrop(224),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
])
attack_imagenet_path = '/local/rcs/yunyun/ImageNet-Data/Attack/'
attack_val_dataset = torchvision.datasets.ImageFolder(root=os.path.join(attack_imagenet_path, 'fgsm_4'), transform=unorm_transforms)
attack_dataset_loader = data.DataLoader(attack_val_dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=8)

In [69]:
print("Adversarial Image & Predicted Label")
save_root_path = '/local/rcs/yunyun/ImageNet-Data/Attack'

    
save_attack_folder = os.path.join(save_root_path, 'pgd_8')
    
correct = 0
total = 0
start = time.time()
    
for i, (images, labels) in enumerate(tqdm(attack_dataset_loader)):
    
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = pretrained_model(images)

    _, pre = torch.max(outputs.data, 1)

    total += images.shape[0]
    correct += (pre == labels).sum()
        
#         img_file_path = os.path.join(save_attack_folder, cls_idx) + '/' + cls_idx + '_' + str(i).zfill(5) + '.JPEG'
        
#         imsave(adv_images.cpu().data[0], img_file_path)

#         imshow(torchvision.utils.make_grid(adv_images.cpu().data, normalize=True), [imagnet_data.classes[i] for i in pre])

print('Total elapsed time (sec): %.2f' % (time.time() - start))
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Adversarial Image & Predicted Label


  0%|          | 0/1563 [00:00<?, ?it/s]

Total elapsed time (sec): 43.59
Robust accuracy: 50.69 %
